###머신러닝 용어정리

 - **입력벡터**: 독립변수
 - **목표벡터**: 종속변수
 - **가중치**: 계수
 - **편향**: 절편(항상 1인 독립변수로 볼수 있음)

###단순 퍼셉트론

- 입력 벡터에 가중치가 반영된 값을 합해서 하나의 출력으로 나오는 것
- 출력과 목표벡터 비교, 손실이 최소가 되도록 가중치 추정
- 출력은 정해진 2개의 값 중 하나가 되는게 보통
- 퍼셉트론: 동그라미 한개

###활성화 함수

- 입력 벡터에 가중치를 적용한 값을 출력으로 변환하는 함수
- 링크함수의 역함수 (로지스틱, 지수함수)와 같은 개념
- 단순 퍼셉트론의 활성화함수는 스텝함수(0 이하면 -1, 0 초과면 1 출력)
- 로지스틱, 항등함수, ReLU함수 등이 있음
- ReLU함수는 x가 0 이하면 0, 0 초과면 x출력

###손실

- 이진분류의 경우 교차 엔트로피 오차, 연속값일 경우 잔차제곱합 사용
- 활성화함수로 로지스틱 함수를 사용하면 손실로 교차 엔트로피 오차를 사용하는 2중 퍼셉트론 로지스틱 회귀모델과 같음
- 활성화함수로 항등함수를 사용하면 손실로 잔차제곱합을 사용하는 2중 퍼셉트론 정규선형모델과 같음

###은닉층

- **입력층**: 입력 벡터가 들어오는 부분
- **출력층**: 예측값이 출력되는 층
- **은닉층(중간층)**: 입력층과 출려층 사이에 있는 층

###신경망

- **신경망(피드포워드 신경망, 다층 퍼셉트론)**: 여러 층의 퍼셉트론으로 구성된 모델
- **딥러닝(심층학습)**: 은닉층을 을린 것
- **CNN(합성곱 신경망)**: 풀링층 같은 특별한 층 포함

모델이 복잡해짐에 따라 발생할 수 있는 과적합 방지하기 위해 **L2 정규화(리지회귀)** 사용

**하이퍼파라미터**: 모델 추정하기 전 가중치 결정 방법, 중간층 개수, 종류, 손실함수 종류 등 바꿔야 되는 요소

###신경망 실습

- 붓꽃의 꽃받침 길이와 너비를 이용해 붓꽃의 종류를 알아내는 예측 모델 구현

In [1]:
import warnings
warnings.filterwarnings(action='ignore')
#수치 계산에 사용하는 라이브러리
import numpy as np
import pandas as pd
import scipy as sp
#선형모델을 추정하는 라이브러리
import statsmodels.formula.api as smf
import statsmodels.api as sm
#다층 퍼셉트론용 애플리케이션
from sklearn.neural_network import MLPClassifier
#샘플 데이터 불러오기
from sklearn.datasets import load_iris
#테스트 데이터와 훈련 데이터 분리
from sklearn.model_selection import train_test_split
#데이터 표준화용 라이브러리
from sklearn.preprocessing import StandardScaler
#표시 자릿수 지정
%precision 3

'%.3f'

데이터 읽기, 다듬기

In [2]:
iris=load_iris()

입력 벡터 이름 확인 - 꽃받침과 꽃잎의 길이와 폭

In [3]:
iris.feature_names

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']

종속변수 확인 - 붓꽃 종류

In [4]:
iris.target_names

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

독립변수 2종류, 종속변수 2종류만 사용할 예정

In [5]:
#독립변수 sepal(꽃받침)만 가져오기
x=iris.data[50:150, 0:2]
#붓꽃의 종류도 2개만 가져오기
y=iris.target[50:150]

print("독립변수 행 수, 열 수: ",x.shape)
print("종속면수 행 수, 열 수: ",y.shape)

독립변수 행 수, 열 수:  (100, 2)
종속면수 행 수, 열 수:  (100,)


In [6]:
#데이터를 훈련 데이터와 테스트 데이터로 분리
x_train, x_test, y_train, y_test=train_test_split(x, y, random_state=2) #데이터를 어떻게 섞을 것인지

print("독립변수 행 수, 열 수: ",x_train.shape)
print("종속면수 행 수, 열 수: ",y_train.shape)

독립변수 행 수, 열 수:  (75, 2)
종속면수 행 수, 열 수:  (75,)


로지스틱 회귀로 분석해보기

In [7]:
y_train[0:10]

array([1, 1, 2, 2, 2, 2, 1, 1, 1, 1])

로지스틱 회귀를 이용해 어떤 종에 속하는지 자동으로 분류

In [8]:
#데이터 정리
#독립변수 데이터프레임
x_train_df=pd.DataFrame(x_train, columns=["sepal_len","sepal_wid"])
#종속변수 데이터프레임
y_train_df=pd.DataFrame({"species":y_train-1}) #종이 1,2로 나오는데 이를 0,1로 표현하기 위해 -1
#데이터프레임 결합
iris_train_df=pd.concat([y_train_df, x_train_df], axis=1)
#결과 출력
print(iris_train_df.head(3))

   species  sepal_len  sepal_wid
0        0        5.7        2.8
1        0        6.6        3.0
2        1        6.1        3.0


In [9]:
#모델로 만들기
#모든 변수를 넣은 모델
logi_mod_full=smf.glm("species~sepal_len+sepal_wid",data=iris_train_df, family=sm.families.Binomial()).fit()
#길이만
logi_mod_len=smf.glm("species~sepal_len",data=iris_train_df, family=sm.families.Binomial()).fit()
#폭만
logi_mod_wid=smf.glm("species~sepal_wid",data=iris_train_df, family=sm.families.Binomial()).fit()
#NULL 모델
logi_mod_null=smf.glm("species~1",data=iris_train_df, family=sm.families.Binomial()).fit()
#AIC 비교
print("full",logi_mod_full.aic.round(3))
print("len",logi_mod_len.aic.round(3))
print("wid",logi_mod_wid.aic.round(3))
print("null",logi_mod_null.aic.round(3))

full 76.813
len 76.234
wid 92.768
null 105.318


길이만 이용한 모델이 모두 이용한 모델보다 AIC 작음 - 길이만 가지고 이용

In [10]:
logi_mod_len.summary().tables[1]

,coef,std err,z,P>|z|,[0.025,0.975]
Intercept,-16.4152,4.000,-4.104,0.000,-24.256,-8.575
sepal_len,2.6478,0.639,4.142,0.000,1.395,3.901


길이가 길수록 2번종일 가능성이 높다

훈련 데이터에 대한 피팅 정확도와 테스트 데이터를 얼마나 정확하게 맞추는지

In [11]:
#데이터 정리
x_test_df=pd.DataFrame(x_test, columns=["sepal_len", "sepal_wid"])
#피팅 예측
logi_fit=logi_mod_len.fittedvalues.round(0)
logi_pred=logi_mod_len.predict(x_test_df).round(0)
#정답 수
true_train=sp.sum(logi_fit==(y_train-1))
true_test=sp.sum(logi_pred==(y_test-1))
#적중률
result_train=true_train/len(y_train)
result_test=true_test/len(y_test)
#결과 출력
print("훈련 데이터 적중률 :",result_train)
print("테스트 데이터 적중률 :",result_test)

훈련 데이터 적중률 : 0.7466666666666667
테스트 데이터 적중률 : 0.68


훈련 데이터와 테스트 데이터에 대해 둘 다 70% 전후로 맞출 수 있다

표준화 - 신경망 이용하려면 리지회귀와 마찬가지로 독립변수 표준화 해야 함

In [18]:
#표준화를 위한 준비
scaler=StandardScaler()
scaler.fit(x_train)
#표준화
x_train_scaled=scaler.transform(x_train)
x_test_scaled=scaler.transform(x_test)

훈련 데이터에 적용한 것과 똑같은 변환을 테스트데이터에도 적용할 수 있다

In [19]:
sp.std(x_train_scaled, axis=0)

array([1., 1.])

In [20]:
sp.std(x_test_scaled, axis=0)

array([0.74 , 0.679])

훈련 데이터의 표준 편차가 1이라고 해서 테스트 데이터의 표준 편차도 1이 되는건 아님

중요한건 훈련 데이터와 테스트 데이터에서 똑같은 변환을 적용했다는 것

신경망

In [22]:
nnet=MLPClassifier(hidden_layer_sizes=(100,100), #은닉층 2층으로 각각의 층에 100개의 유닛 배치 (입출력층까지 합쳐 총 4층)
                   alpha=0.07, #정규화 강도 지정
                   max_iter=10000, #파라미터 추정 위해 최대 몇번이나 반복계산 할지
                   random_state=0) #난수 고정
nnet.fit(x_train_scaled, y_train)
#정답률
print("훈련 데이터 적중률: ", nnet.score(x_train_scaled, y_train))
print("테스트 데이터 적중률: ", nnet.score(x_test_scaled, y_test))

훈련 데이터 적중률:  0.8933333333333333
테스트 데이터 적중률:  0.68
